In [1]:
%tensorflow_version 2.x
import tensorflow as tf
import os
import numpy as np
import cv2 as cv
import keras.backend as K

Colab only includes TensorFlow 2.x; %tensorflow_version has no effect.


Model Building

In [2]:
def conv_block(inputs,conv_type,kernel,kernel_size,strides,padding='same',relu=True):
    if conv_type == 'ds':
        x = tf.keras.layers.SeparableConv2D(kernel,kernel_size,padding=padding,strides=strides)(inputs)
    else:
        x = tf.keras.layers.Conv2D(kernel,kernel_size,padding=padding,strides=strides)(inputs)
    x = tf.keras.layers.BatchNormalization()(x)
    if relu:
        x = tf.keras.activations.relu(x)
    return x

def _res_bottleneck(inputs,filters,kernel,t,s,r=False):
  tchannel = tf.keras.backend.int_shape(inputs)[-1]*t
  x = conv_block(inputs,'conv',tchannel,(1,1),strides=(1,1))
  x = tf.keras.layers.DepthwiseConv2D(kernel,strides=(s,s),depth_multiplier=1,padding='same')(x)
  x = tf.keras.layers.BatchNormalization()(x)
  x = tf.keras.activations.relu(x)
  x = conv_block(x,'conv',filters,(1,1),strides=(1,1),padding='same',relu=False)
  if r:
      x = tf.keras.layers.add([x,inputs])
  return x

def bottleneck_block(inputs,filters,kernel,t,strides,n):
    x = _res_bottleneck(inputs,filters,kernel,t,strides)
    for i in range(1,n):
        x = _res_bottleneck(x,filters,kernel,t,1,True)
    return x

def pyramid_pooling_block(input_tensor,bin_sizes):
    concat_list = [input_tensor]
#     w = 64
#     h = 32
    w = 32
    h = 64
    for bin_size in bin_sizes:
        x = tf.keras.layers.AveragePooling2D(pool_size=(w//bin_size,h//bin_size),strides=(w//bin_size,h//bin_size))(input_tensor)
        x = tf.keras.layers.Conv2D(128,3,2,padding='same')(x)
        x = tf.keras.layers.Lambda(lambda x:tf.image.resize(x,(w,h)))(x)
        concat_list.append(x)
    return tf.keras.layers.concatenate(concat_list)

input_layer = tf.keras.layers.Input(shape=(512,1024,3),name='input_layer') #(2048,1024,3)
# input_layer = tf.keras.layers.BatchNormalization()(input_layer)
lds_layer = conv_block(input_layer,'conv',32,(3,3),strides=(2,2))
lds_layer = conv_block(lds_layer,'ds',48,(3,3),strides=(2,2))
lds_layer = conv_block(lds_layer,'ds',64,(3,3),strides=(1,1)) 
gfe_layer = bottleneck_block(lds_layer,64,(3,3),t=6,strides=2,n=3) 
gfe_layer = bottleneck_block(gfe_layer,96,(3,3),t=6,strides=2,n=3)
gfe_layer = bottleneck_block(gfe_layer,128,(3,3),t=6,strides=1,n=3) 
gfe_layer = pyramid_pooling_block(gfe_layer,[2,4,6,8]) 
ff_layer1 = conv_block(lds_layer,'conv',128,(1,1),padding='same',strides=(1,1),relu=False)
ff_layer2 = tf.keras.layers.UpSampling2D((4,4))(gfe_layer)
ff_layer2 = tf.keras.layers.DepthwiseConv2D(128,strides=(1,1),depth_multiplier=1,padding='same')(ff_layer2)
ff_layer2 = tf.keras.layers.BatchNormalization()(ff_layer2)
ff_layer2 = tf.keras.activations.relu(ff_layer2)
ff_layer2 = tf.keras.layers.Conv2D(128,1,1,padding='same',activation=None)(ff_layer2)
ff_final = tf.keras.layers.add([ff_layer1,ff_layer2])
ff_final = tf.keras.layers.BatchNormalization()(ff_final)
ff_final = tf.keras.activations.relu(ff_final)
classifier = tf.keras.layers.SeparableConv2D(128,(3,3),padding='same',strides=(1,1),name="DSConv1_classifier")(ff_final)
classifier = tf.keras.layers.BatchNormalization()(classifier)
classifier = tf.keras.activations.relu(classifier)

classifier = tf.keras.layers.SeparableConv2D(128,(3,3),padding='same',strides=(1,1),name="DSConv2_classifier")(classifier)
classifier = tf.keras.layers.BatchNormalization()(classifier)
classifier = tf.keras.activations.relu(classifier)

classifier = conv_block(classifier,'conv',9,(1,1),strides=(1,1),padding='same',relu=True) # classes number

classifier = tf.keras.layers.Dropout(0.1)(classifier)

classifier = tf.keras.layers.UpSampling2D((4,4))(classifier) #(8,8)
classifier = tf.keras.activations.softmax(classifier)

Loss Function

In [3]:
# def dice_coef(y_true, y_pred, smooth=1):
#     y_true_f = K.flatten(y_true)
#     y_pred_f = K.flatten(y_pred)
#     intersection = K.sum(y_true_f * y_pred_f)
#     print(intersection)
#     return (2. * intersection + smooth) / (K.sum(y_true_f) + K.sum(y_pred_f) + smooth)

# def dice_coef_loss(y_true, y_pred):
#     return 1 - dice_coef(y_true, y_pred)

def dice_coef(y_true, y_pred, smooth=1):
    
    y_true = y_true[...,1:]
    y_pred = y_pred[...,1:]
    
    numerator = 2 * K.sum(y_true * y_pred,axis=[0,1,2]) + smooth 
    denominator = K.sum(y_true,axis=[0,1,2]) + K.sum(y_pred,axis=[0,1,2]) + smooth
    
    return K.mean( numerator / denominator )

def dice_coef_loss(y_true, y_pred):
    return 1 - dice_coef(y_true, y_pred)

Compiling Model

In [4]:
fast_scnn = tf.keras.Model(inputs=input_layer, outputs=classifier, name = 'Fast_SCNN')
optimizer = tf.keras.optimizers.SGD(momentum=0.9,lr=0.32) 
weights = '/content/drive/MyDrive/fast_scnn/checkpointweight0.h5'
fast_scnn.load_weights(weights)

#metrics = tf.keras.metrics.MeanIoU(9, name=None, dtype=None)
fast_scnn.compile(loss=dice_coef_loss,optimizer=optimizer,metrics=[dice_coef])

/usr/local/lib/python3.7/dist-packages/keras/optimizer_v2/gradient_descent.py:102: UserWarning: The `lr` argument is deprecated, use `learning_rate` instead.
  super(SGD, self).__init__(name, **kwargs)


Data Pipeline

In [5]:
image_path_og = os.path.join('drive','MyDrive','CITY','Resized','images')
label_path_og = os.path.join('drive','MyDrive','CITY','Resized','labels_limited')
file_order_path = os.path.join('drive','MyDrive','CITY','File Order')
  
def load_data(cat):
  x = []
  y = []
  
  img_order_path = os.path.join(file_order_path,'img_{}_file_order.npy'.format(cat))
  lab_order_path = os.path.join(file_order_path,'label_{}_file_order.npy'.format(cat))
  img_order = np.load(img_order_path)
  lab_order = np.load(lab_order_path)
  
  for img,lab in zip(img_order,lab_order):
    image_path = os.path.join(image_path_og,img)
    label_path = os.path.join(label_path_og,lab)
    m = img.split('leftImg8bit')
    l = lab.split('gtFine_labelIds')
    if m==l:
      x.append(image_path)
      y.append(label_path)
    else:
      print('[ERR]Data Mismatch')
  if cat == 'train':
    return x[720:],y[720:]
  else:
    return x,y

def read_image(path):
  x = cv.imread(path)
  x = cv.cvtColor(x,cv.COLOR_BGR2RGB)
  return x

def read_mask(path):
    x = cv.imread(path,cv.IMREAD_UNCHANGED)
    # x = np.expand_dims(x,2)
    x = (np.arange(9) == x[...,None]).astype(np.int8) #number_of_classes
    return x

def preprocess(x, y):
  def f(x, y):
    x = x.decode()
    y = y.decode()

    x = read_image(x)
    y = read_mask(y)

    # return x, y
    return x, y.astype('float32')

  images, masks = tf.numpy_function(f, [x, y], [tf.uint8, tf.float32])
  # images, masks = tf.numpy_function(f, [x, y], [tf.uint8, tf.float32])
  
  images.set_shape([512, 1024, 3])
  masks.set_shape([512, 1024, 9])

  return images, masks

def tf_dataset(x, y, batch=7):
  dataset = tf.data.Dataset.from_tensor_slices((x, y))
  dataset = dataset.map(preprocess)
  dataset = dataset.batch(batch)
  dataset = dataset.prefetch(2)
  return dataset

val_images, val_masks = load_data('val')
val_dataset = tf_dataset(val_images, val_masks)

train_images, train_masks = load_data('train')
train_dataset = tf_dataset(train_images, train_masks)

Training

In [ ]:
filepath = os.path.join('drive','MyDrive','fast_scnn','checkpointweight0.h5')
checkpoint = tf.keras.callbacks.ModelCheckpoint(
    filepath,
    monitor='accuracy',
    verbose=1,
    save_best_only=False,
    save_weights_only=True,
    save_freq=60,
)
H = fast_scnn.fit(train_dataset, epochs = 1, validation_data = (val_dataset),callbacks=[checkpoint])
fast_scnn.save_weights(os.path.join('drive','MyDrive','fast_scnn','weight_last.h5'))
model_path = os.path.join('drive','MyDrive','fast_scnn','fast_scnn.h5')
fast_scnn.save(model_path)

  1/323 [..............................] - ETA: 11:09:51 - loss: 0.5359 - dice_coef: 0.4641